In [1]:
!pip install -qU crewai langchain_openai 'crewai[tools]' google-search-results

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.1/337.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.0/633.0 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
from getpass import getpass

# Ask the user to enter the API keys (they won't show up on screen)
os.environ["GEMINI_API_KEY"] = getpass("Enter your Gemini API key: ")
os.environ["SERPER_API_KEY"] = getpass("Enter your Serper API key: ")

Enter your Gemini API key: ··········
Enter your Serper API key: ··········


###Initialize Models

In [4]:
from langchain_openai import ChatOpenAI
from crewai import LLM

Gemini = LLM(
    model="gemini/gemini-2.0-flash",)


#gpt4o = ChatOpenAI(model = "gpt-4o")

### AI Web Scraping Agent 🔍

In [5]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

# Initialize tools
search_tool = SerperDevTool()

def create_web_scraping_agent(url, target_data):
    # Define the Web Scraping Agent
    scraping_agent = Agent(
        role="Data Collector",
        goal=f"Collect and analyze {target_data} from {url}",
        backstory="An expert data analyst trained to gather and process web information efficiently",
        verbose=True,
        tools=[search_tool],
        llm=Gemini
    )

    # Define the Data Processing Agent
    processing_agent = Agent(
        role="Data Processor",
        goal=f"Process and structure the collected {target_data}",
        backstory="A data processing specialist who organizes and formats collected information into useful insights",
        verbose=True,
        tools=[search_tool],
        llm=Gemini
    )

    # Define the Data Collection Task
    scraping_task = Task(
        description=f"Search and collect {target_data} related to {url}",
        expected_output=f"Raw collected data about {target_data} from {url}",
        agent=scraping_agent
    )

    # Define the Data Processing Task
    processing_task = Task(
        description=f"Process and structure the collected {target_data} into a clear format",
        expected_output=f"A structured and cleaned summary of {target_data}",
        agent=processing_agent
    )

    # Create and Run the Crew
    crew = Crew(
        agents=[scraping_agent, processing_agent],
        tasks=[scraping_task, processing_task],
        verbose=True,
        process=Process.sequential
    )

    result = crew.kickoff(inputs={"url": url, "target_data": target_data})
    return result

/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


### Execute the agent


In [6]:
url = input("Enter the website URL to scrape: ")
target_data = input("Enter the type of data to extract (e.g., product prices, news headlines): ")
scraping_result = create_web_scraping_agent(url, target_data)
print("Web Scraping Results:")
print(scraping_result)

Enter the website URL to scrape: https://www.ilovepdf.com/
Enter the type of data to extract (e.g., product prices, news headlines): what does this do


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 975b1965-5905-47b8-9204-3dc121f36690                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Collector                                                                                          │
│                                                                                                                 │
│  Task: Search and collect what does this do related to https://www.ilovepdf.com/                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Processor                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  iLovePDF provides a range of online tools and functionalities for managing PDF files. These include:           │
│                                                                                                                 │
│  *   **Conversion:** PDF to Word, PDF to Excel, PDF to PowerPoint, PDF to JPG, and vice versa.                  │
│  *   **Editing:** Adding text, images, watermarks, annotations, page numbers, and other modifications; editing  │
│  existing text, changing fonts, colors and sizes.                                                               │
│  *   **Merging:** Combining multiple PDF files.                                                                 │
│  *   **Splitting:** Extracting pages from a PDF.                                                                │
│  *   **Compression:** Reducing the size of PDF files.                                                           │
│  *   **Protection:** Adding passwords, restricting editing and printing.                                        │
│  *   **Organization:** Rotating, deleting, and rearranging pages.                                               │
│  *   **Other:** Unlocking, repairing PDFs, OCR (Optical Character Recognition).                                 │
│                                                                                                                 │
│  iLovePDF offers a Web App, Desktop App, and a Mobile App to edit, convert, and manage your PDF files online    │
│  and offline with ease. It also has cloud storage integration.                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 108a3e33-da65-4754-ae4c-2e6d4e4fe590                                                                     │
│  Agent: Data Processor                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 975b1965-5905-47b8-9204-3dc121f36690                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: iLovePDF provides a range of online tools and functionalities for managing PDF files. These      │
│  include:                                                                                                       │
│                                                                                                                 │
│  *   **Conversion:** PDF to Word, PDF to Excel, PDF to PowerPoint, PDF to JPG, and vice versa.                  │
│  *   **Editing:** Adding text, images, watermarks, annotations, page numbers, and other modifications; editing  │
│  existing text, changing fonts, colors and sizes.                                                               │
│  *   **Merging:** Combining multiple PDF files.                                                                 │
│  *   **Splitting:** Extracting pages from a PDF.                                                                │
│  *   **Compression:** Reducing the size of PDF files.                                                           │
│  *   **Protection:** Adding passwords, restricting editing and printing.                                        │
│  *   **Organization:** Rotating, deleting, and rearranging pages.                                               │
│  *   **Other:** Unlocking, repairing PDFs, OCR (Optical Character Recognition).                                 │
│                                                                                                                 │
│  iLovePDF offers a Web App, Desktop App, and a Mobile App to edit, convert, and manage your PDF files online    │
│  and offline with ease. It also has cloud storage integration.                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Web Scraping Results:
iLovePDF provides a range of online tools and functionalities for managing PDF files. These include:

*   **Conversion:** PDF to Word, PDF to Excel, PDF to PowerPoint, PDF to JPG, and vice versa.
*   **Editing:** Adding text, images, watermarks, annotations, page numbers, and other modifications; editing existing text, changing fonts, colors and sizes.
*   **Merging:** Combining multiple PDF files.
*   **Splitting:** Extracting pages from a PDF.
*   **Compression:** Reducing the size of PDF files.
*   **Protection:** Adding passwords, restricting editing and printing.
*   **Organization:** Rotating, deleting, and rearranging pages.
*   **Other:** Unlocking, repairing PDFs, OCR (Optical Character Recognition).

iLovePDF offers a Web App, Desktop App, and a Mobile App to edit, convert, and manage your PDF files online and offline with ease. It also has cloud storage integration.
